# Multithreading with Numba
Aufgrund der CPU-Einschränkungen bei der Ausführung von Notebooks auf Binder werden die Vorteile von Multithreading unberechenbar sein. Der Code in diesem Notebook wird auf Binder ausgeführt, aber für vernünftige Benchmarks sollten Sie dieses Notebook herunterladen und auf Ihrem eigenen System ausführen.*

Numba unterstützt mehrere Ansätze für Multithreading:

* Automatisches Multithreading von Array-Ausdrücken und -Reduktionen
* Explizites Multithreading von Schleifen mit `prange()`
* Externes Multithreading mit Tools wie concurrent.futures oder Dask.

Die ersten beiden Optionen nutzen den *ParallelAccelerator*-Optimierungspass (von Intel beigesteuert) in Numba. ParallelAccelerator wird nur auf 64-Bit-Plattformen unterstützt und ist für Python 2.7 unter Windows nicht verfügbar. Es ist auch nur wirksam, wenn im Nopython-Modus kompiliert wird.

In [12]:
import numpy as np
import numba
from numba import jit

## Automatic Multithreading

NumPy-Array-Ausdrücke weisen eine beträchtliche implizite Parallelität auf, da Operationen unabhängig über die Eingabeelemente gesendet werden. ParallelAccelerator kann diese Parallelität erkennen und automatisch auf mehrere Threads verteilen. Alles, was wir tun müssen, ist den Parallelisierungsdurchlauf mit `parallel=True` zu ​​aktivieren:

In [13]:
SQRT_2PI = np.sqrt(2 * np.pi)

@jit(nopython=True, parallel=True)
def gaussians(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result / SQRT_2PI / n

In [14]:
means = np.random.uniform(-1, 1, size=1000000)
widths = np.random.uniform(0.1, 0.3, size=1000000)

gaussians(0.4, means, widths)

array([1.60820340e-06, 1.41785675e-06, 1.23636617e-07, ...,
       2.91865528e-08, 4.55294303e-08, 2.88343262e-13])

Um den Effekt mehrerer CPUs zu sehen, können wir den Fall vergleichen, in dem ParallelAccelerator deaktiviert ist. In Anbetracht dessen, dass Decorators Funktionen sind, die andere Funktionen umwandeln, können wir "jit" als Funktion aufrufen:

In [27]:
gaussians_nothread = jit(nopython=True)(gaussians.py_func)

%timeit gaussians_nothread(0.4, means, widths)


7.54 ms ± 595 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit gaussians(0.4, means, widths)

3.84 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can also compare the performance to the uncompiled NumPy array evaluation using the `.py_func` attribute to get the original Python function:

In [29]:
%timeit gaussians.py_func(0.4, means, widths) # compare to pure NumPy

29.8 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Das Leistungsverhältnis hängt von der Anzahl der CPUs in Ihrem System ab, aber die Multithread-Version ist definitiv schneller als die Single-Thread-Version.

ParallelAccelerator kann auch mit Reduktionen umgehen:

In [30]:
@jit(nopython=True, parallel=True)
def kde(x, means, widths):
    '''Return the value of gaussian kernels.
    
    x - location of evaluation
    means - array of kernel means
    widths - array of kernel widths
    '''
    n = means.shape[0]
    result = np.exp( -0.5 * ((x - means) / widths)**2 ) / widths
    return result.mean() / SQRT_2PI

kde_nothread = jit(nopython=True)(kde.py_func)

In [31]:
%timeit kde_nothread(0.4, means, widths)
%timeit kde(0.4, means, widths)

7.73 ms ± 717 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.15 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Multithreading with `prange()`

Es gibt andere Situationen, in denen Sie Multithreading möchten, aber keinen einfachen Array-Ausdruck haben. In diesen Fällen zeigt die Verwendung von "prange()" in einer for-Schleife ParallelAccelerator an, dass dies eine Schleife ist, bei der jede Iteration unabhängig von der anderen ist und parallel ausgeführt werden kann.

Beispielsweise möchten wir möglicherweise viele Monte-Carlo-Versuche hintereinander ausführen:

In [35]:
import random

# Serial version
@jit(nopython=True)
def monte_carlo_pi_serial(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

# Parallel version
@jit(nopython=True, parallel=True)
def monte_carlo_pi_parallel(nsamples):
    acc = 0
    # Only change is here
    for i in numba.prange(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

def monte_carlo_pi_serial_o(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

Beachten Sie, dass `prange()` die Reduktionsvariable `acc` automatisch Thread-sicher für Sie handhabt. Wir verlassen uns auch auf Numba, um den Zufallszahlengenerator in jedem Thread unabhängig voneinander automatisch zu initialisieren.

Sie können auch jeden Thread in einem `prange()` veranlassen, ein separates Element in einem Ausgabe-Array zu modifizieren, aber allgemeinere Race-Bedingungen werden nicht automatisch von ParallelAccelerator aufgelöst. Vorsichtig sein!

Mal sehen, wie schnell diese beiden Implementierungen sind:

In [32]:
%time monte_carlo_pi_serial(int(4e8))


CPU times: total: 4.8 s
Wall time: 4.84 s


3.1416874

In [33]:
%time monte_carlo_pi_parallel(int(4e8))

CPU times: total: 12.8 s
Wall time: 1.91 s


3.14158536

In [36]:
%time monte_carlo_pi_serial_o(int(4e8))

CPU times: total: 6min 37s
Wall time: 6min 40s


3.14162044

In [37]:
print(int(4e8))

400000000


The parallel version saturates all the CPUs once the initial compilation finishes.

### External Multithreading

Manchmal befindet sich Ihr Threading-System vollständig außerhalb von Numba. Möglicherweise verwenden Sie „concurrent.futures“, um Funktionen in mehreren Threads auszuführen, oder ein paralleles Framework wie [Dask](http://dask.pydata.org/). Für diese Situationen möchten Sie ParallelAccelerator nicht verwenden, aber zulassen, dass die von Numba kompilierte Funktion gleichzeitig in verschiedenen Threads ausgeführt wird.

Dazu soll die Numba-Funktion während der Ausführung die Global Interpreter Lock (GIL) freigeben. Dies kann mit der Option `nogil=True` zu ​​`@jit` gemacht werden.

Machen wir unser Monte-Carlo-Beispiel noch einmal, aber mit Dask. Beachten Sie, dass Numba weiterhin die Initialisierung separater Zufallszahlengenerator-Seeds auf jedem Thread handhabt, wie es bei ParallelAccelerator der Fall war.

In [38]:
import dask
import dask.delayed

@jit(nopython=True, nogil=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

print(monte_carlo_pi(int(4e8)))

delayed_monte_carlo_pi = dask.delayed(monte_carlo_pi)

3.14163503


Parallel execution:

In [39]:
%%time
futures = [delayed_monte_carlo_pi(int(4e8)) for i in range(4)]
results = dask.compute(futures)[0]
print(sum(results)/4) # average resuts

3.1416007775000003
CPU times: total: 34.6 s
Wall time: 9.09 s


Serial execution

In [23]:
%%time
futures = [delayed_monte_carlo_pi(int(4e8)) for i in range(4)]
results = dask.compute(futures, num_workers=1)[0]
print(sum(results)/4) # average resuts

3.14161225
CPU times: total: 25.8 s
Wall time: 26 s
